In [4]:
code = {
    "name": "commands",
    "body": [
        {
            "name": "repeat-n",
            "count": 10,
            "body": [
                {
                    "name": "gpio-write",
                    "pin": 16,
                    "value": 1
                },
                {
                    "name": "wait-sec",
                    "duration": 1
                },
                {
                    "name": "gpio-write",
                    "pin": 16,
                    "value": 0
                },
                {
                    "name": "wait-sec",
                    "duration": 1
                }
            ]
        }, {
            "name": "raw",
            "commands": [
                "ping 0",
                "ident"
            ]
        }
    ]
}

In [5]:
from proc_comp import parser

p = parser.parse(code)

#print(p)
print(p.pprint())

SeqExp([
  RepeatExp((U32) 10, [
    SeqExp([
      ProcSetExp(gpio_mode[16], (U8) 1)
      ProcSetExp(gpio_value[16], (U8) 1)
    ])
    WaitTimeExp((U32) 1)
    SeqExp([
      ProcSetExp(gpio_mode[16], (U8) 1)
      ProcSetExp(gpio_value[16], (U8) 0)
    ])
    WaitTimeExp((U32) 1)
  ])
  RawCSH([
    ping 0
    ident
  ])
])


In [6]:
from proc_comp.codegen.codegen import CodeGen

G = CodeGen()
G.code_gen(p)


['proc del 10 12',
 'proc new',
 'proc set _p_uint32[1] 0',
 'proc set _p_uint32[0] 10',
 'proc call 11',
 'ping 0',
 'ident',
 'proc push 10 12',
 'proc del 11 12',
 'proc new',
 'proc ifelse _p_uint32[1] < _p_uint32[0]',
 'proc call 12',
 'proc noop',
 'proc push 11 12',
 'proc del 12 12',
 'proc new',
 'proc set gpio_mode[16] 1',
 'proc set gpio_value[16] 1',
 'proc unop time idt _p_uint32[2]',
 'proc set _p_uint32[3] 1',
 'proc binop _p_uint32[2] + _p_uint32[3] _p_uint32[2]',
 'proc block time >= _p_uint32[2]',
 'proc set gpio_mode[16] 1',
 'proc set gpio_value[16] 0',
 'proc unop time idt _p_uint32[2]',
 'proc set _p_uint32[3] 1',
 'proc binop _p_uint32[2] + _p_uint32[3] _p_uint32[2]',
 'proc block time >= _p_uint32[2]',
 'proc unop _p_uint32[1] ++ _p_uint32[1]',
 'proc call 11',
 'proc push 12 12',
 'proc run 10 12']